In [1]:
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.svm import SVR
from xgboost import XGBRegressor
from datetime import date, datetime
from sklearn.impute import SimpleImputer
from IPython.core.display import display, HTML
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import Lasso, LinearRegression, Ridge
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
%matplotlib inline
warnings.filterwarnings('ignore')
pd.pandas.set_option('display.max_rows', None)
pd.pandas.set_option('display.max_columns', None)
display(HTML('<style>.container{width : 100% ! important;}</style>'))
print('Libraries imported successfully!!')

Libraries imported successfully!!


In [2]:
df = pd.read_csv('IndianPremierLeague.csv')
df.shape

(193468, 38)

In [3]:
df['runs_last_5_overs'] = 0

In [4]:
for i in range(1, df.shape[0]):
    if df['over'][i] <= 4:
        df['runs_last_5_overs'][i] = df['ball_by_ball_score'][i]

In [5]:
for i in df.index:
    if df['over'][i] > 4:
        if df['id'][i] == df['id'][i-30]:
            if df['inning'][i] == df['inning'][i-30]:
                df['runs_last_5_overs'][i] = df['ball_by_ball_score'][i] - df['ball_by_ball_score'][i-30]

In [7]:
df['wickets_last_5_overs'] = 0

In [8]:
df.head()

,id,date,city,venue,1st_team,neutral_venue,2nd_team,toss_winner,toss_decision,inning,batting_team,bowling_team,batsman,non_striker,bowler,over,ball,batsman_runs,extra_runs,total_runs,non_boundary,is_wicket,dismissal_kind,player_dismissed,fielder,extras_type,ball_by_ball_score,winner,result,result_margin,player_of_match,eliminator,method,umpire1,umpire2,final_score,total_wickets,ball_by_ball_wicket,runs_last_5_overs,wickets_last_5_overs
0,335982,18-04-2008,Bangalore,M Chinnaswamy Stadium,Royal Challengers Bangalore,0,Kolkata Knight Riders,Royal Challengers Bangalore,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,BB McCullum,P Kumar,0,1,0,1,1,0,0,NaN,NaN,NaN,legbyes,1,Kolkata Knight Riders,runs,140.0,BB McCullum,N,NaN,Asad Rauf,RE Koertzen,222,3,0,0,0
1,335982,18-04-2008,Bangalore,M Chinnaswamy Stadium,Royal Challengers Bangalore,0,Kolkata Knight Riders,Royal Challengers Bangalore,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,2,0,0,0,0,0,NaN,NaN,NaN,NaN,1,Kolkata Knight Riders,runs,140.0,BB McCullum,N,NaN,Asad Rauf,RE Koertzen,222,3,0,1,0
2,335982,18-04-2008,Bangalore,M Chinnaswamy Stadium,Royal Challengers Bangalore,0,Kolkata Knight Riders,Royal Challengers Bangalore,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,3,0,1,1,0,0,NaN,NaN,NaN,wides,2,Kolkata Knight Riders,runs,140.0,BB McCullum,N,NaN,Asad Rauf,RE Koertzen,222,3,0,2,0
3,335982,18-04-2008,Bangalore,M Chinnaswamy Stadium,Royal Challengers Bangalore,0,Kolkata Knight Riders,Royal Challengers Bangalore,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,4,0,0,0,0,0,NaN,NaN,NaN,NaN,2,Kolkata Knight Riders,runs,140.0,BB McCullum,N,NaN,Asad Rauf,RE Koertzen,222,3,0,2,0
4,335982,18-04-2008,Bangalore,M Chinnaswamy Stadium,Royal Challengers Bangalore,0,Kolkata Knight Riders,Royal Challengers Bangalore,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,5,0,0,0,0,0,NaN,NaN,NaN,NaN,2,Kolkata Knight Riders,runs,140.0,BB McCullum,N,NaN,Asad Rauf,RE Koertzen,222,3,0,2,0


In [9]:
for i in range(1, df.shape[0]):
    if df['over'][i] <= 4:
        df['wickets_last_5_overs'][i] = df['ball_by_ball_wicket'][i]

In [10]:
for i in df.index:
    if df['over'][i] > 4:
        if df['id'][i] == df['id'][i-30]:
            if df['inning'][i] == df['inning'][i-30]:
                df['wickets_last_5_overs'][i] = df['ball_by_ball_wicket'][i] - df['ball_by_ball_wicket'][i-30]

In [11]:
for i in df.index:
    if df['over'][i] > 4:
        if df['id'][i] != df['id'][i-1]:
            df['wickets_last_5_overs'][i-1] = df['ball_by_ball_wicket'][i-1] - df['ball_by_ball_wicket'][i-31]

In [12]:
for i in df.index:
    if df['over'][i] > 4:
        if df['id'][i] == df['id'][i-1]:
            if df['inning'][i] != df['inning'][i-1]:
                df['wickets_last_5_overs'][i-1] = df['ball_by_ball_wicket'][i-1] - df['ball_by_ball_wicket'][i-31]

In [13]:
df.head(300)

,id,date,city,venue,1st_team,neutral_venue,2nd_team,toss_winner,toss_decision,inning,batting_team,bowling_team,batsman,non_striker,bowler,over,ball,batsman_runs,extra_runs,total_runs,non_boundary,is_wicket,dismissal_kind,player_dismissed,fielder,extras_type,ball_by_ball_score,winner,result,result_margin,player_of_match,eliminator,method,umpire1,umpire2,final_score,total_wickets,ball_by_ball_wicket,runs_last_5_overs,wickets_last_5_overs
0,335982,18-04-2008,Bangalore,M Chinnaswamy Stadium,Royal Challengers Bangalore,0,Kolkata Knight Riders,Royal Challengers Bangalore,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,SC Ganguly,BB McCullum,P Kumar,0,1,0,1,1,0,0,NaN,NaN,NaN,legbyes,1,Kolkata Knight Riders,runs,140.0,BB McCullum,N,NaN,Asad Rauf,RE Koertzen,222,3,0,0,0
1,335982,18-04-2008,Bangalore,M Chinnaswamy Stadium,Royal Challengers Bangalore,0,Kolkata Knight Riders,Royal Challengers Bangalore,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,2,0,0,0,0,0,NaN,NaN,NaN,NaN,1,Kolkata Knight Riders,runs,140.0,BB McCullum,N,NaN,Asad Rauf,RE Koertzen,222,3,0,1,0
2,335982,18-04-2008,Bangalore,M Chinnaswamy Stadium,Royal Challengers Bangalore,0,Kolkata Knight Riders,Royal Challengers Bangalore,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,3,0,1,1,0,0,NaN,NaN,NaN,wides,2,Kolkata Knight Riders,runs,140.0,BB McCullum,N,NaN,Asad Rauf,RE Koertzen,222,3,0,2,0
3,335982,18-04-2008,Bangalore,M Chinnaswamy Stadium,Royal Challengers Bangalore,0,Kolkata Knight Riders,Royal Challengers Bangalore,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,4,0,0,0,0,0,NaN,NaN,NaN,NaN,2,Kolkata Knight Riders,runs,140.0,BB McCullum,N,NaN,Asad Rauf,RE Koertzen,222,3,0,2,0
4,335982,18-04-2008,Bangalore,M Chinnaswamy Stadium,Royal Challengers Bangalore,0,Kolkata Knight Riders,Royal Challengers Bangalore,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,5,0,0,0,0,0,NaN,NaN,NaN,NaN,2,Kolkata Knight Riders,runs,140.0,BB McCullum,N,NaN,Asad Rauf,RE Koertzen,222,3,0,2,0
5,335982,18-04-2008,Bangalore,M Chinnaswamy Stadium,Royal Challengers Bangalore,0,Kolkata Knight Riders,Royal Challengers Bangalore,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,6,0,0,0,0,0,NaN,NaN,NaN,NaN,2,Kolkata Knight Riders,runs,140.0,BB McCullum,N,NaN,Asad Rauf,RE Koertzen,222,3,0,2,0
6,335982,18-04-2008,Bangalore,M Chinnaswamy Stadium,Royal Challengers Bangalore,0,Kolkata Knight Riders,Royal Challengers Bangalore,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,P Kumar,0,7,0,1,1,0,0,NaN,NaN,NaN,legbyes,3,Kolkata Knight Riders,runs,140.0,BB McCullum,N,NaN,Asad Rauf,RE Koertzen,222,3,0,3,0
7,335982,18-04-2008,Bangalore,M Chinnaswamy Stadium,Royal Challengers Bangalore,0,Kolkata Knight Riders,Royal Challengers Bangalore,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,Z Khan,1,1,0,0,0,0,0,NaN,NaN,NaN,NaN,3,Kolkata Knight Riders,runs,140.0,BB McCullum,N,NaN,Asad Rauf,RE Koertzen,222,3,0,3,0
8,335982,18-04-2008,Bangalore,M Chinnaswamy Stadium,Royal Challengers Bangalore,0,Kolkata Knight Riders,Royal Challengers Bangalore,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,Z Khan,1,2,4,0,4,0,0,NaN,NaN,NaN,NaN,7,Kolkata Knight Riders,runs,140.0,BB McCullum,N,NaN,Asad Rauf,RE Koertzen,222,3,0,7,0
9,335982,18-04-2008,Bangalore,M Chinnaswamy Stadium,Royal Challengers Bangalore,0,Kolkata Knight Riders,Royal Challengers Bangalore,field,1,Kolkata Knight Riders,Royal Challengers Bangalore,BB McCullum,SC Ganguly,Z Khan,1,3,4,0,4,0,0,NaN,NaN,NaN,NaN,11,Kolkata Knight Riders,runs,140.0,BB McCullum,N,NaN,Asad Rauf,RE Koertzen,222,3,0,11,0


In [14]:
df.to_csv('IndianPremierLeague.csv', index = None)